<a href="https://colab.research.google.com/github/WahyuKhairi06/DeepLearning-WahyuKhairi-2311531009/blob/main/Praktikum%2010/Deep_Belief_Network_(DBN)_dan_Capsule_Network_(CapsNet).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Deep Belief Netowrk***

In [ ]:
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from tensorflow.keras.datasets import mnist



In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# reshape dan scaling biar mirip openml
# X_train = X_train.reshape((60000, 784)) / 255.0
# X_test = X_test.reshape((10000, 784)) / 255.0
# reshape (70000, 784)
X_train = X_train.reshape(-1, 784).astype('float32')
X_test  = X_test.reshape(-1, 784).astype('float32')

# normalisasi 0–1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
X = X.astype('float32')
X = MinMaxScaler().fit_transform(X)
y = y.astype('int')


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
rbm = BernoulliRBM(n_components=256, learning_rate=0.06, batch_size=10, n_iter=10, verbose=1)
rbm2 = BernoulliRBM(n_components=128, learning_rate=0.06, batch_size=10, n_iter=10, verbose=1)
logistic = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='multinomial')

In [7]:
# 3. DBN sebagai pipeline RBM + Logistic Regression
dbn = Pipeline(steps=[('rbm', rbm), ('logistic', logistic)])
dbn.fit(X[:60000], y[:60000])

[BernoulliRBM] Iteration 1, pseudo-likelihood = -89.77, time = 22.91s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -86.75, time = 22.26s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -84.89, time = 20.48s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -82.75, time = 23.51s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -84.83, time = 22.72s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -81.58, time = 20.95s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -83.95, time = 21.03s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -81.36, time = 22.05s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -79.02, time = 20.29s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -80.67, time = 21.34s


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Pipeline(steps=[('rbm', BernoulliRBM(learning_rate=0.06, verbose=1)),
                ('logistic',
                 LogisticRegression(max_iter=1000, multi_class='multinomial'))])

In [8]:
# 4. Evaluasi
X_eval, y_eval = fetch_openml('mnist_784', version=1, return_X_y=True)
X_eval = X_eval.astype('float32')
X_eval = MinMaxScaler().fit_transform(X_eval)
y_eval = y_eval.astype('int')

score = dbn.score(X_eval[60000:], y_eval[60000:])
print(f"Akurasi DBN pada data uji: {score * 100:.2f}%")

Akurasi DBN pada data uji: 96.39%


***Capsule Network***

In [9]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import MinMaxScaler

# 1️ Load & preprocess MNIST
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
X = X.astype('float32')
X = MinMaxScaler().fit_transform(X)
y = y.astype('int64').to_numpy()

# 2️ Reshape ke format citra (N, 1, 28, 28)
X = X.reshape(-1, 1, 28, 28)

# 3️ Split 60k train, 10k test
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]

# 4️ Buat DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 64

train_ds = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
test_ds  = TensorDataset(torch.from_numpy(X_test),  torch.from_numpy(y_test))

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False)

print("Using device:", device)
print(f"Train samples: {len(train_ds)} | Test samples: {len(test_ds)}")


Using device: cuda
Train samples: 60000 | Test samples: 10000


In [10]:
# Cell 2: CapsNet + CapsuleLayer (einsum routing)
import torch.nn as nn
import torch.nn.functional as F

class CapsuleLayer(nn.Module):
    def __init__(self, num_capsules, num_route_nodes, in_channels, out_channels, num_iterations=3):
        super().__init__()
        self.num_capsules = num_capsules
        self.num_route_nodes = num_route_nodes
        self.num_iterations = num_iterations
        # route weights: [num_capsules, num_route_nodes, in_channels, out_channels]
        self.route_weights = nn.Parameter(torch.randn(num_capsules, num_route_nodes, in_channels, out_channels) * 0.1)

    def squash(self, tensor, dim=-1):
        norm = (tensor ** 2).sum(dim=dim, keepdim=True)
        scale = norm / (1.0 + norm)
        return scale * tensor / torch.sqrt(norm + 1e-8)

    def forward(self, x):
        # x: [batch, num_route_nodes, in_channels]
        # u_hat: [batch, num_capsules, num_route_nodes, out_channels]
        u_hat = torch.einsum('bin,cino->bcio', x, self.route_weights)
        batch_size = u_hat.size(0)
        b = torch.zeros(batch_size, self.num_capsules, self.num_route_nodes, device=u_hat.device)
        for i in range(self.num_iterations):
            c = F.softmax(b, dim=1)                             # coupling coeffs over capsules
            s = (c.unsqueeze(-1) * u_hat).sum(dim=2)            # [batch, num_capsules, out_channels]
            v = self.squash(s, dim=-1)
            if i < self.num_iterations - 1:
                agreement = (u_hat * v.unsqueeze(2)).sum(dim=-1)
                b = b + agreement
        return v  # [batch, num_capsules, out_channels]


class CapsNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 256, kernel_size=9, stride=1)        # -> [B,256,20,20]
        self.primary_caps = nn.Conv2d(256, 8*32, kernel_size=9, stride=2)  # -> [B,8*32,6,6]
        self.digit_caps = CapsuleLayer(num_capsules=10, num_route_nodes=32*6*6, in_channels=8, out_channels=16)

    def forward(self, x):
        x = F.relu(self.conv1(x))                      # [B,256,20,20]
        x = self.primary_caps(x)                       # [B,8*32,6,6]
        batch_size = x.size(0)
        x = x.view(batch_size, 32*6*6, 8)              # [B, num_route_nodes, in_channels]
        x = self.digit_caps(x)                         # [B, 10, 16]
        lengths = torch.sqrt((x ** 2).sum(dim=-1) + 1e-8)  # [B,10]
        return lengths, x

# instantiate
model = CapsNet().to(device)
print(model)


CapsNet(
  (conv1): Conv2d(1, 256, kernel_size=(9, 9), stride=(1, 1))
  (primary_caps): Conv2d(256, 256, kernel_size=(9, 9), stride=(2, 2))
  (digit_caps): CapsuleLayer()
)


In [11]:
# Cell 3: margin loss (Sabour) + optimizer + helpers
import torch.optim as optim

def margin_loss(lengths, labels, m_plus=0.9, m_minus=0.1, lambda_val=0.5):
    one_hot = F.one_hot(labels, num_classes=lengths.size(1)).float().to(lengths.device)
    left = F.relu(m_plus - lengths) ** 2
    right = F.relu(lengths - m_minus) ** 2
    loss = one_hot * left + lambda_val * (1.0 - one_hot) * right
    return loss.sum(dim=1).mean()

def accuracy(lengths, labels):
    preds = lengths.argmax(dim=1)
    return (preds == labels).float().mean().item()

optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [12]:
num_epochs = 5

for epoch in range(1, num_epochs+1):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        lengths, recon = model(images)
        loss = margin_loss(lengths, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    avg_loss = running_loss / len(train_loader.dataset)

    # EVALUATION
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            lengths, recon = model(images)    # FIX: 2 output
            preds = lengths.argmax(dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    test_acc = correct / total * 100
    print(f"Epoch {epoch:02d} | Train Loss: {avg_loss:.4f} | Test Acc: {test_acc:.2f}%")


Epoch 01 | Train Loss: 3.6412 | Test Acc: 10.85%
Epoch 02 | Train Loss: 3.6444 | Test Acc: 10.10%
Epoch 03 | Train Loss: 3.4806 | Test Acc: 9.82%
Epoch 04 | Train Loss: 3.2819 | Test Acc: 9.82%
Epoch 05 | Train Loss: 3.2241 | Test Acc: 9.20%
